In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.7 MB/s eta 0:00:00


# Load Model 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('microsoft/DialoGPT-small')

In [3]:
# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:1


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: 
>> User:2


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: 
>> User:3


...

DialoGPT: 
>> User:4


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: 
>> User:5


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: 


# Load Data

In [5]:
import pandas as pd

huberman_data = pd.read_csv('/content/drive/MyDrive/YouTube-Data/Huberman/Huberman-transcripts-v5.csv')
print('length of data: ', len(huberman_data))
huberman_data.head(10)

length of data:  194294


,episode,guest,time,text
0,1,NaN,0.38,- Welcome to the Huberman Lab Podcast where we...
1,1,NaN,3.75,and science-based tools for everyday life. (up...
2,1,NaN,9.53,I'm Andrew Huberman and I'm a professor of\nne...
3,1,NaN,13.72,at Stanford School of Medicine. For today's po...
4,1,NaN,17.85,the parts list of the nervous system. Now that...
5,1,NaN,22.07,but these are the bits and pieces that togethe...
6,1,NaN,26.53,"about your experience of life,\nfrom what you ..."
7,1,NaN,31.61,and what you accomplish from the day you're bo...
8,1,NaN,35.72,That parts list is really incredible because i...
9,1,NaN,41.277,that really provides a window\ninto all sorts ...


In [6]:
# Eliminate the \n in text 
huberman_data['text'] = huberman_data['text'].str.replace('\n', ' ')
huberman_data.head(10)

,episode,guest,time,text
0,1,NaN,0.38,- Welcome to the Huberman Lab Podcast where we...
1,1,NaN,3.75,and science-based tools for everyday life. (up...
2,1,NaN,9.53,I'm Andrew Huberman and I'm a professor of neu...
3,1,NaN,13.72,at Stanford School of Medicine. For today's po...
4,1,NaN,17.85,the parts list of the nervous system. Now that...
5,1,NaN,22.07,but these are the bits and pieces that togethe...
6,1,NaN,26.53,"about your experience of life, from what you t..."
7,1,NaN,31.61,and what you accomplish from the day you're bo...
8,1,NaN,35.72,That parts list is really incredible because i...
9,1,NaN,41.277,that really provides a window into all sorts o...


In [7]:
contexted = []
n = 7
text = huberman_data['text']

for i in range(n, len(text)):
  row = []
  prev = i - 1 - n
  for j in range(i, prev, -1):
    row.append(text[j])
  contexted.append(row)
contexted[:5]

[["and what you accomplish from the day you're born until the day you die.",
  'about your experience of life, from what you think about to what you feel, what you imagine,',
  'but these are the bits and pieces that together make up everything',
  'the parts list of the nervous system. Now that might sound boring,',
  "at Stanford School of Medicine. For today's podcast we're going to talk about",
  "I'm Andrew Huberman and I'm a professor of neurobiology and ophthalmology",
  'and science-based tools for everyday life. (upbeat guitar music)',
  '- Welcome to the Huberman Lab Podcast where we discuss science'],
 ['That parts list is really incredible because it has a history associated with it',
  "and what you accomplish from the day you're born until the day you die.",
  'about your experience of life, from what you think about to what you feel, what you imagine,',
  'but these are the bits and pieces that together make up everything',
  'the parts list of the nervous system. Now th

In [8]:
columns = ['response', 'context'] + ['context/'+str(i) for i in range(n-1)]
columns

['response',
 'context',
 'context/0',
 'context/1',
 'context/2',
 'context/3',
 'context/4',
 'context/5']

In [9]:
df = pd.DataFrame.from_records(contexted, columns=columns)
df.head()

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
0,and what you accomplish from the day you're bo...,"about your experience of life, from what you t...",but these are the bits and pieces that togethe...,the parts list of the nervous system. Now that...,at Stanford School of Medicine. For today's po...,I'm Andrew Huberman and I'm a professor of neu...,and science-based tools for everyday life. (up...,- Welcome to the Huberman Lab Podcast where we...
1,That parts list is really incredible because i...,and what you accomplish from the day you're bo...,"about your experience of life, from what you t...",but these are the bits and pieces that togethe...,the parts list of the nervous system. Now that...,at Stanford School of Medicine. For today's po...,I'm Andrew Huberman and I'm a professor of neu...,and science-based tools for everyday life. (up...
2,that really provides a window into all sorts o...,That parts list is really incredible because i...,and what you accomplish from the day you're bo...,"about your experience of life, from what you t...",but these are the bits and pieces that togethe...,the parts list of the nervous system. Now that...,at Stanford School of Medicine. For today's po...,I'm Andrew Huberman and I'm a professor of neu...
3,So I'm going to share with you the parts list ...,that really provides a window into all sorts o...,That parts list is really incredible because i...,and what you accomplish from the day you're bo...,"about your experience of life, from what you t...",but these are the bits and pieces that togethe...,the parts list of the nervous system. Now that...,at Stanford School of Medicine. For today's po...
4,through the lens of some of those other aspect...,So I'm going to share with you the parts list ...,that really provides a window into all sorts o...,That parts list is really incredible because i...,and what you accomplish from the day you're bo...,"about your experience of life, from what you t...",but these are the bits and pieces that togethe...,the parts list of the nervous system. Now that...


In [10]:
from sklearn.model_selection import train_test_split
trn_df, val_df = train_test_split(df, test_size=0.1)
print('Length of Training Data: ', len(trn_df))
print('Length of Val Data: ', len(val_df))

Length of Training Data:  174858
Length of Val Data:  19429


In [11]:
trn_df = trn_df.sample(15000)
val_df = val_df.sample(1500)

In [12]:
trn_df[:3]

,response,context,context/0,context/1,context/2,context/3,context/4,context/5
106967,"I mean, in reality it hasn't gotten shorter ov...",if you're bringing your wrist closer to your s...,"like tendons and ligaments. And for instance,","by way of changing the length of the muscle, a...","And when muscles contract, they are able to mo...","these neurons, onto the muscles causes the mus...","But at the neuromuscular junction, the release...","and does other things in your brain, mainly it..."
172223,extra factors total stress load hydration sleep,out what you can't but let's say in science yo...,who like through any training study just won't...,responders the bell curve in the middle of the...,all right so if you do that you can see a beau...,not the group average that's very important to...,report those individual data because when you ...,better how many got worse how many maybe chang...
31205,"and stable, and it's not very specific. You're...",it's got this almost medieval lack of specific...,Now it's a very clinically safe and stable pro...,- It doesn't look like one fluid there last se...,"- I mean, it is. These days, it's much more cl...",where you're causing a brainwide- - Which look...,"have the least specificity, electroconvulsive ...","And actually, it's a frustrating aspect of psy..."


In [49]:
import os
import pickle
import logging
from transformers import PreTrainedTokenizer, MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pathlib import Path
from typing import Dict, List, Tuple

logger = logging.getLogger(__name__)

def construct_convo(row, tokenizer):
  print(row)
  print(tokenizer.encode(row[0]))
  print(tokenizer.encode(row[-1]))
  conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row])) 
  print(conv) 
  flatten = lambda l: [item for sublist in l for item in sublist]
  conv = flatten(conv)
  print(conv)
  return conv

class ConversationDataset(Dataset):
  def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):
    block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

    directory = args.cache_dir
    cached_features_file = os.path.join(
        directory, args.model_type + "_cached_lm_" + str(block_size)
    )

    if os.path.exists(cached_features_file) and not args.overwrite_cache:
      logger.info("Loading features from cached file %s", cached_features_file)
      with open(cached_features_file, "rb") as handle:
          self.examples = pickle.load(handle)
    else:
      logger.info("Creating features from dataset file at %s", directory)

      self.examples = []
      for _, row in df.iterrows():
          conv = construct_convo(row, tokenizer)
          self.examples.append(conv)
      
      logger.info("Saving features into cached file %s", cached_features_file)
      with open(cached_features_file, "wb") as handle:
        pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

  def __len__(self):
    return len(self.examples)

  def __getitem__(self, item):
    return torch.tensor(self.examples[item], dtype=torch.long)

In [50]:
for _, row in trn_df.iterrows():
  conv = construct_convo(row, tokenizer)
  break

response     I mean, in reality it hasn't gotten shorter ov...
context      if you're bringing your wrist closer to your s...
context/0        like tendons and ligaments. And for instance,
context/1    by way of changing the length of the muscle, a...
context/2    And when muscles contract, they are able to mo...
context/3    these neurons, onto the muscles causes the mus...
context/4    But at the neuromuscular junction, the release...
context/5    and does other things in your brain, mainly it...
Name: 106967, dtype: object
[40, 1612, 11, 287, 3950, 340, 5818, 470, 7891, 12238, 4045, 11, 340, 338, 655, 13413, 12238, 11, 286, 1781, 13]
[392, 857, 584, 1243, 287, 534, 3632, 11, 8384, 340, 338, 2950, 287, 2962, 290, 3241, 13]
[[392, 857, 584, 1243, 287, 534, 3632, 11, 8384, 340, 338, 2950, 287, 2962, 290, 3241, 13, 50256], [1537, 379, 262, 11943, 296, 385, 10440, 35037, 11, 262, 2650, 286, 25174, 2645, 354, 14453, 422, 777, 16384, 4778, 11, 50256], [27218, 16890, 11, 4291, 262, 12749, 5

In [26]:
# Caching and storing of data/checkpoints
import re
import random
import glob
import shutil
import numpy as np 

def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
   return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

def set_seed(args):
   random.seed(args.seed)
   np.random.seed(args.seed)
   torch.manual_seed(args.seed)
   if args.n_gpu > 0:
       torch.cuda.manual_seed_all(args.seed)


def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
   ordering_and_checkpoint_path = []


   glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))


   for path in glob_checkpoints:
       if use_mtime:
           ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
       else:
           regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
           if regex_match and regex_match.groups():
               ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))


   checkpoints_sorted = sorted(ordering_and_checkpoint_path)
   checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
   return checkpoints_sorted


def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
   if not args.save_total_limit:
       return
   if args.save_total_limit <= 0:
       return


   # Check if we should delete older checkpoint(s)
   checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
   if len(checkpoints_sorted) <= args.save_total_limit:
       return

   number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
   checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
   for checkpoint in checkpoints_to_be_deleted:
       logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
       shutil.rmtree(checkpoint)

In [27]:
from transformers import PreTrainedModel, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm, trange

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )
    print("Batch size: ", args.train_batch_size)
    print("Data Loader length: ", len(train_dataloader))

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))
    # add_special_tokens_(model, tokenizer)


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                print("Batch shape: ", batch.shape)
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [40]:
# Main runner
from transformers import AutoConfig

def main(df_trn, df_val):
    args = Args()
    
    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
        os.path.exists(args.output_dir)
        and os.listdir(args.output_dir)
        and args.do_train
        and not args.overwrite_output_dir
        and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir='/content/'+args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir='/content/'+args.cache_dir)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir='/content/'+args.cache_dir,
    )
    model.to(args.device)
    
    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)
        print(train_dataset)
        print("Train Dataset Length: ", len(train_dataset))
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    # Saving best-practices: if you use save_pretrained for the model and tokenizer, you can reload them using from_pretrained()
    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)
        # Save a trained model, configuration and tokenizer using `save_pretrained()`.
        # They can then be reloaded using `from_pretrained()`
        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

In [41]:
class Args():
    def __init__(self):
        self.output_dir = 'HubermanGPT-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 4
        self.per_gpu_eval_batch_size = 4
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'


args = Args()

In [42]:
main(trn_df, val_df)

Train Dataset Length:  15000
Batch size:  4
Data Loader length:  3750


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3750 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Iteration:   0%|          | 0/3750 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3750 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/375 [00:00<?, ?it/s]

{'perplexity_': tensor(13.8289)}

In [46]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('HubermanGPT-small')

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000, 
        pad_token_id=tokenizer.eos_token_id, 
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature = 0.8,
                                      )

    # pretty print last ouput tokens from bot
    print("HubermanBot: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

>> User:what is the best health protocol?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


HubermanBot: Well, if you're somebody who has a very, very, extremely high baseline of testosterone,
>> User:tell me more?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


HubermanBot: I mean, I have a very high baseline, but I'm not a fan of taking anything that's too high.
>> User:good to know i guess


...

HubermanBot: what I'm talking about here is that there are a number of people out there that are interested in optimizing their hormones
>> User:how can i optimize them?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


HubermanBot: In fact, we'll talk about how to optimize your hormones in a little bit.
>> User:okay ....


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


HubermanBot: !!!I want to make sure I'm clear, I want to talk about optimizing hormone levels


In [47]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [48]:
model.push_to_hub('HubermanGPT-small-v1')
tokenizer.push_to_hub('HubermanGPT-small-v1')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jasondubon/HubermanGPT-small-v1/commit/822efc0eba820c136d9943fcf6c4db687ea81213', commit_message='Upload tokenizer', commit_description='', oid='822efc0eba820c136d9943fcf6c4db687ea81213', pr_url=None, pr_revision=None, pr_num=None)